# ==============================================
# 2_data_preprocessing.ipynb
# Datenbereinigung, Tokenisierung & Split-Erstellung
# ==============================================

# Zelle 1: Importiere benötigte Bibliotheken
import os
import pandas as pd
import re
import json
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize

# Stelle sicher, dass NLTK-Tokendaten vorhanden sind
nltk.download('punkt')

# Definiere Pfade zu den Rohdaten und zum Speicherort der verarbeiteten Daten
RAW_DATA_DIR = "../data/raw"
PROCESSED_DATA_DIR = "../data/processed"

# Zelle 2: Rohdaten laden
# Beispiel: Artikel oder längere Texte, die zusammengefasst werden sollen
raw_file_path = os.path.join(RAW_DATA_DIR, "articles_raw.csv")
df = pd.read_csv(raw_file_path)
print("Anzahl Rohtexte:", len(df))
display(df.head())

# Zelle 3: Datenbereinigung
def clean_text(text):
    """
    Entfernt unerwünschte HTML-Tags, überflüssige Leerzeichen und trimmt den Text.
    Optional wird der Text in Kleinbuchstaben konvertiert.
    """
    if not isinstance(text, str):
        text = str(text)
    # Entferne HTML-Tags (z. B. <p>, <br/>)
    text = re.sub(r"<.*?>", "", text)
    # Ersetze mehrere Leerzeichen durch ein einzelnes Leerzeichen
    text = re.sub(r"\s+", " ", text)
    # Trim und Lowercase
    return text.strip().lower()

df["cleaned_text"] = df["text"].apply(clean_text)
# Entferne Texte, die zu kurz sind (z.B. weniger als 100 Zeichen)
df = df[df["cleaned_text"].str.len() > 100]
print("\nBeispiel bereinigter Texte:")
display(df["cleaned_text"].head())

# Zelle 4: Tokenisierung
def tokenize_text(text):
    """
    Zerlegt den Text in Tokens.
    Hier verwenden wir den einfachen NLTK-Word-Tokenizer.
    """
    tokens = word_tokenize(text)
    return tokens

df["tokens"] = df["cleaned_text"].apply(tokenize_text)
print("\nBeispiel tokenisierter Texte:")
display(df["tokens"].head())

# Zelle 5: Vokabular erstellen
from collections import Counter

all_tokens = []
for tokens in df["tokens"]:
    all_tokens.extend(tokens)

token_freqs = Counter(all_tokens)
print(f"Anzahl eindeutiger Tokens: {len(token_freqs)}")

# Beschränke das Vokabular ggf. auf die häufigsten Tokens, um Rauschen zu reduzieren.
VOCAB_SIZE = 10000  # oder setze VOCAB_SIZE = len(token_freqs), wenn du alle behalten möchtest
most_common_tokens = token_freqs.most_common(VOCAB_SIZE)

# Definiere Sondertokens
special_tokens = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
word2id = {}
idx = 0
for token in special_tokens:
    word2id[token] = idx
    idx += 1

for token, freq in most_common_tokens:
    if token not in word2id:
        word2id[token] = idx
        idx += 1

print(f"Vokabulargröße (inkl. Sondertokens): {len(word2id)}")
# Erstelle auch das inverse Mapping
id2word = {v: k for k, v in word2id.items()}

# Zelle 6: Tokens in IDs umwandeln
def tokens_to_ids(token_list, word2id, unk_id=word2id["<UNK>"]):
    """
    Wandelt eine Liste von Tokens in ihre entsprechenden IDs um.
    Tokens, die nicht im Vokabular gefunden werden, erhalten die <UNK>-ID.
    """
    return [word2id[t] if t in word2id else unk_id for t in token_list]

df["token_ids"] = df["tokens"].apply(lambda tokens: tokens_to_ids(tokens, word2id))
print("\nBeispiel für token_ids:")
display(df["token_ids"].head())

# Zelle 7: Sequenzen vorbereiten und Datensatz-Split
# Für Summarization trainierst du meist ein Seq2Seq-Modell.
# Hier nehmen wir an, dass der gesamte Text der Input ist und 
# die Zusammenfassung (falls vorhanden) als Target genutzt wird.
# Falls keine Zusammenfassungen vorhanden sind, kann der Text als Input
# genutzt werden und später durch ein abstraktives Modell zusammengefasst werden.
# Beispiel: Wir splitten nur den Originaltext in Train/Val, da das Ziel die Zusammenfassung ist,
# die im Training generiert werden soll.

# Für Educational-Zwecke nutzen wir hier einen einfachen Split:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Zelle 8: Speichern der verarbeiteten Daten und des Vokabulars
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Speichere das Vokabular als JSON
vocab_path = os.path.join(PROCESSED_DATA_DIR, "vocab.json")
with open(vocab_path, "w", encoding="utf-8") as f:
    json.dump(word2id, f, ensure_ascii=False)
print("Vokabular gespeichert in:", vocab_path)

# Speichere die DataFrames als CSV-Dateien
train_path = os.path.join(PROCESSED_DATA_DIR, "train.csv")
val_path = os.path.join(PROCESSED_DATA_DIR, "val.csv")
test_path = os.path.join(PROCESSED_DATA_DIR, "test.csv")

train_df.to_csv(train_path, index=False)
val_df.to_csv(val_path, index=False)
test_df.to_csv(test_path, index=False)

print("Daten gespeichert:")
print("Train CSV:", train_path)
print("Val CSV:", val_path)
print("Test CSV:", test_path)

# Zelle 9: Ausblick
print("""
Die Daten wurden erfolgreich bereinigt, tokenisiert und in Trainings-, Validierungs- und Test-Sets aufgeteilt.
Als nächster Schritt können wir das AutoSummary-Modell (Seq2Seq-LSTM oder Transformer) trainieren.
Sieh dir dazu das Notebook 3_training_demo.ipynb oder das Skript src/train.py an.
""")
